In [1]:
import os

In [2]:

%pwd

'e:\\datascience\\projects\\End_to_End-Chest_classification\\research'

In [3]:

os.chdir("../")

In [4]:
%pwd

'e:\\datascience\\projects\\End_to_End-Chest_classification'

In [ ]:
MLFLOW_TRACKING_URI=https://dagshub.com/Mohsin678/End_to_End-Chest_classification-.mlflow \
MLFLOW_TRACKING_USERNAME=Mohsin678 \
MLFLOW_TRACKING_PASSWORD=82201297cc14f4a9196a58d5a292142006635238 \
python script.py

In [15]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Mohsin678/End_to_End-Chest_classification-.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="mohsin678"
os.environ["MLFLOW_TRACKING_PASSWORD"]="82201297cc14f4a9196a58d5a292142006635238"



In [16]:

import tensorflow as tf


In [17]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [18]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    Path_of_model:Path
    training_data:Path
    all_params:dict
    mlflow_uri:str
    params_image_size:list
    params_batch_size:int

In [19]:
from chestclassification.constants import *
from chestclassification.utils.common import read_yaml,create_directories,save_json

In [20]:
class ConfigurationManager:
    def __init__(
            self,
            config_path = config_file_path,
            params_filepath = params_file_path):

            self.config =read_yaml(config_path)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])


    def get_evaluation_config(self)  -> EvaluationConfig:
          eval_config = EvaluationConfig(
                Path_of_model="Artifacts/training/model.h5",
                training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
                mlflow_uri = "https://dagshub.com/Mohsin678/End_to_End-Chest_classification-.mlflow",
                all_params=self.params,
                params_image_size=self.params.IMAGE_SIZE,
                params_batch_size=self.params.BATCH_SIZE
          )

          return eval_config

In [21]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [23]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.Path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss":self.score[0],"accuracy":self.score[1]}
        save_json(path =Path("scores:json"),data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_score = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1] })


            if tracking_url_type_score !="file":

                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [25]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-05-08 18:03:47,899: INFO:  common: yamle_file: config\config.yaml loaded successfully]
[2024-05-08 18:03:47,903: INFO:  common: yamle_file: params.yaml loaded successfully]
[2024-05-08 18:03:47,906: INFO:  common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 761s 110s/step - loss: 14.2996 - accuracy: 0.4314
[2024-05-08 18:16:31,652: INFO:  common: json file saved at: scores:json]


2024/05/08 18:16:35 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-05-08 18:16:40,317: WARNING:  save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Home\AppData\Local\Temp\tmpz58lxt4q\model\data\model\assets
[2024-05-08 18:16:43,063: INFO:  builder_impl: Assets written to: C:\Users\Home\AppData\Local\Temp\tmpz58lxt4q\model\data\model\assets]


g:\anaconda\envs\cancer\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2024/05/08 18:19:01 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
